# Base case




In [14]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import random
import scipy
import sklearn
from scipy import optimize
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
import sys
import time

In [29]:
# initialize the cost model for winter
costMdl_x = MLPRegressor(max_iter=1000, solver='sgd',learning_rate_init = 0.01, hidden_layer_sizes = (10,10,))
# initialize parameters of the DLC sequence x
x = [[0,0,0]]
# define search limits for DLC parameters 
bounds_x = [[0,3],[0,3],[0,3]]
# define function to be used in updating the x
def wrapperx(x):
    xp = costMdl_x.predict([[x[0],x[1],x[2]]])
    return xp

# initialize the cost model for summer
costMdl_y = MLPRegressor(max_iter=1000, solver='sgd',learning_rate_init = 0.01, hidden_layer_sizes = (10,10,))
# initialize parameters of the DLC sequence y
y = [[0,0,0,0,0,0]]
# define search limits for DLC parameters 
bounds_y = [[0,3],[0,3],[0,3],[0,3],[0,3],[0,3]]
def wrappery(y):
    yp = costMdl_y.predict([[y[0],y[1],y[2],y[3],y[4],y[5]]])
    return yp

a = 8.7*0.01  #$/kWh  off_peak_price
b = 12.2*0.01  #$/kWh  mid_peak_price
c = 18.2*0.01  #$/kWh  on_peak_price
winter_prices = {0:a ,1:a ,2:a ,3:a ,4:a ,5:a ,6:a ,7:c ,8:c ,9:c ,10:c ,11:b ,12:b ,13:b ,14:b ,15:b ,16:b ,17:c ,18:c ,19:a ,20:a ,21:a ,22:a ,23:a }
summer_prices = {0:a ,1:a ,2:a ,3:a ,4:a ,5:a ,6:a ,7:b ,8:b ,9:b ,10:b ,11:c ,12:c ,13:c ,14:c ,15:c ,16:c ,17:b ,18:b ,19:a ,20:a ,21:a ,22:a ,23:a }
J_2_kWh = 1/3600000

discomfort_totalizer = 0
energy_totalizer = 0
cost_x = 0 # winter daily cost
cost_y = 0 # summer daily cost

exploration_x_count = 0
exploration_y_count = 0

store_x = []
store_y = []
cost_winter = []
cost_summer = []
        
def my_callback_function(state):
    global discomfort_totalizer, energy_totalizer, x, y, cost_x, cost_y, exploration_x_count, exploration_y_count
    envNum = api.exchange.current_environment_num(state)
    if envNum == 3: # if sizing done
        tSetHtg_hndl = api.exchange.get_actuator_handle(state, "Schedule:Constant", "Schedule Value", "tSetHtg")
        tSetClg_hndl = api.exchange.get_actuator_handle(state, "Schedule:Constant", "Schedule Value", "tSetClg")
        elect_hndl = api.exchange.get_meter_handle(state,"Electricity:Facility")
        tIn_office_hndl = api.exchange.get_variable_handle(state,"Zone Mean Air Temperature","Office")
        tIn_lobby_hndl = api.exchange.get_variable_handle(state,"Zone Mean Air Temperature","Lobby")

        # exchange information with EnergyPlus
        meter_reading = api.exchange.get_meter_value(state,elect_hndl)
         
        tIn_office = api.exchange.get_variable_value(state,tIn_office_hndl)
        tIn_lobby = api.exchange.get_variable_value(state,tIn_lobby_hndl)
        month = api.exchange.month(state) 
        time_of_day = api.exchange.hour(state)
        day_of_week = api.exchange.day_of_week(state)
        day_of_month = api.exchange.day_of_month(state)
        
        # calculate price based on time of day & season
        if ((month > 10) or (month < 5)) and (day_of_week > 1 and day_of_week < 7):  # winter, weekday
            energy_dollars = meter_reading * J_2_kWh * winter_prices[time_of_day]
        elif ((month > 4) and (month < 11)) and (day_of_week > 1 and day_of_week < 7): # summer, weekday
            energy_dollars = meter_reading * J_2_kWh * summer_prices[time_of_day]
        else:  # weekend
            energy_dollars = meter_reading * J_2_kWh * a
                
        
        api.exchange.set_actuator_value(state, tSetHtg_hndl, 18)
        api.exchange.set_actuator_value(state, tSetClg_hndl, 27)

        if (day_of_week > 1 and day_of_week < 7):
            if (time_of_day > 7 and time_of_day < 17):
                api.exchange.set_actuator_value(state, tSetHtg_hndl, 21)
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24)

        # actuate setpoints based on winter DLC parameter array x
        if ((month > 10) or (month < 5)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day > 7 - x[0][0]) and (time_of_day < 7): # preheat winter off-peak
                api.exchange.set_actuator_value(state, tSetHtg_hndl, 21 + x[0][1])
            if (time_of_day > 7) and (time_of_day < 11): # setback winter on-peak
                api.exchange.set_actuator_value(state, tSetHtg_hndl, 21 - x[0][2])

        # actuate setpoints based on summer parameter array y   
        if ((month > 4) and (month < 11)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day > 7 - y[0][0]) and (time_of_day < 7): # precool summer off-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24 - y[0][1])
            if (time_of_day > 7) and (time_of_day < 7 + y[0][2]): # setup summer mid-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24 + y[0][3])
            if (time_of_day > 7 + y[0][2]) and (time_of_day < 11): # precool summer mid-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24 - y[0][4])
            if (time_of_day > 11) and (time_of_day < 17): # setup summer on-peak
                api.exchange.set_actuator_value(state, tSetClg_hndl, 24 + y[0][5])

        # at the end of the day update cost_x to update cost model for winter
        if ((month > 10) or (month < 5)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day < 23):
                cost_x = 0
                energy_totalizer = energy_totalizer + energy_dollars
                if (time_of_day > 8 and time_of_day < 17):
                    if ((tIn_office < 20) or (tIn_office > 25) or (tIn_lobby < 20) or (tIn_lobby > 25)):
                        discomfort_totalizer = discomfort_totalizer + 1
            elif (cost_x == 0):
                cost_x = (energy_totalizer) * (1 + (discomfort_totalizer > 4)) + 0.0000001
                discomfort_totalizer = 0
                energy_totalizer = 0
                
                cost_winter.append(cost_x)


        # at the end of the day update cost_y to update cost model for summer
        if ((month > 4) and (month < 11)) and (day_of_week > 1 and day_of_week < 7):
            if (time_of_day < 23):
                cost_y = 0
                energy_totalizer = energy_totalizer + energy_dollars
                if (time_of_day > 8 and time_of_day < 17):
                    if ((tIn_office < 20) or (tIn_office > 25) or (tIn_lobby < 20) or (tIn_lobby > 25)):
                        discomfort_totalizer = discomfort_totalizer + 1
            elif (cost_y == 0):
                cost_y = (energy_totalizer) * (1 + (discomfort_totalizer > 4)) + 0.0000001
                discomfort_totalizer = 0
                energy_totalizer = 0
                
                cost_summer.append(cost_y)
        
        store_x.append(x)
        store_y.append(y)
        
        

In [30]:
import sys
sys.path.insert(0, 'C:\EnergyPlusV23-1-0')  # add E-Plus directory to path to be able to import API
from pyenergyplus.api import EnergyPlusAPI #import EnergyPlus library
api = EnergyPlusAPI()
state = api.state_manager.new_state() 
api.runtime.clear_callbacks()
api.functional.clear_callbacks()
# energyplus model calling point, callback function
api.runtime.callback_begin_system_timestep_before_predictor(state,my_callback_function)
# run EPlus
epwFile = 'ON_OTTAWA-INTL-ONT_716280_19.epw'
idfFile = 'baseline.idf'
# -x short form to run expandobjects for HVACtemplates. see EnergyPlusEssentials.pdf p16
cmd_args = ['-w',epwFile, '-d', 'out_base','-x',idfFile]
api.runtime.run_energyplus(state,cmd_args)

0

In [34]:
df = pd.DataFrame([cost_winter, cost_summer]).transpose()
df.columns = ["cost_winter","cost_summer"]
df.to_csv('base_case_data.csv', index=False)